## Visualizing eigen modes with ipywidgets and pysurfer

In [1]:
# testing for visualization within the notebook cells, won't update with slider tho have to call test_plot3d()
# from mayavi import mlab
# mlab.init_notebook('x3d')
# mlab.test_plot3d()

In [1]:
from ipywidgets import interactive, widgets, fixed
import sys
sys.path.append("../..")

import numpy as np
import pandas as pd
from surfer import Brain as surfBrain
import os
import nibabel as nib

from spectrome.brain import Brain
from spectrome.utils import functions, path

In [2]:
em_brain = Brain.Brain() # creates empty brain object
hcp_dir = path.get_sibling_path('data') # grabs path to data folder one folder above (sibling)
em_brain.add_connectome(hcp_dir) # adds connectivity matrix and distance matrix to brain object
em_brain.reorder_connectome(em_brain.connectome, em_brain.distance_matrix) # <- talk to Pablo to see if this is necessary
#em_brain.reorder_connectome(em_brain.connectome, em_brain.distance_matrix)
em_brain.bi_symmetric_c()
em_brain.reduce_extreme_dir()

# Define frequency range of interest
fmin = 2 # 2Hz - 45Hz signal range, filter for this with hbp
fmax = 45
fvec = np.linspace(fmin,fmax,40)

In [3]:
# LaPlacian Function:
def get_complex_laplacian(C, D, w, speed = 10, num_ev = 5):
    """ Extract complex laplacian based on frequency "omega", returns number of eigen
        vectors. This function sorts eigen vectors by ascending order, meaning the sorting
        begins with the eigen vectors associated with smallest absolute value of eigen values. 
        
        Args:
        - C (array): connectome
        - D (array): Distance matrix for C
        - speed (int): default 10 m/s, transmission velocity
        - omega (float): 
        - num_ev (int): number of eigen vectors you want as output
        Output:
        - L: complex laplacian
        - Vvec: eigen vectors
    """
    alpha = 1
    nroi = C.shape[0]
    rowdegree = np.transpose(np.sum(C,axis=1))
    coldegree = np.sum(C,axis=0)
    
    qind = rowdegree + coldegree < 0.2*np.mean(rowdegree + coldegree)
    rowdegree[qind] = np.inf
    coldegree[qind] = np.inf
    
    Tau = 0.001*D/speed
    Cc = C*np.exp(-1j*Tau*w)
    
    # Eigen decomposition
    L1 = 0.8*np.identity(nroi)
    L2 = np.divide(1,np.sqrt(np.multiply(rowdegree,coldegree))+np.spacing(1)) #diag(1./(sqrt(rowdegree.*coldegree)+eps));
    L = L1 - np.matmul(np.diag(L2),Cc)
    # how many eigs to pick?
    use_smalleigs = False
    numsmalleigs = np.round(2/3*C.shape[0]) #2/3
    if use_smalleigs is True:
        K = numsmalleigs
        K = K.astype(int)
    else:
        K = nroi
    # decomposition with linalg.eig    
    d, v = np.linalg.eig(L)
    # sorting in ascending & absolute value
    eig_ind = np.argsort(np.abs(d))
    eig_vec = v[:,eig_ind]
    abseiv = np.abs(eig_vec[:,0])
    sorted_Evals = d[eig_ind]
    #abseval = np.abs(eig_val)
    
    ev = np.transpose(sorted_Evals[0:K])
    Vv = eig_vec[0:K]
    Vvec = np.asarray(Vv)
    
    selected_Evec = []
    for k in np.arange(0, num_ev):
        abs_Vvec = np.abs(Vvec[:,k])
        selected_Evec.append(abs_Vvec)
    
    selected_Evec = np.asarray(selected_Evec)
    return L, selected_Evec, sorted_Evals

## function to normalize the eigen modes..?
def normalize_eigs(selected_Evec):
    norm_eigs = np.zeros(selected_Evec.shape)
    nroi = selected_Evec.shape[0]
    for i in np.arange(0,nroi):
        vdata = np.maximum(selected_Evec[i,:], 
                   np.mean(selected_Evec[i,:])-np.std(selected_Evec[i,:]))
        vdata = vdata - np.amin(vdata)
        
        vdata = np.minimum(vdata, np.mean(vdata)+np.std(vdata))
        vdata = vdata/np.amax(vdata)
        norm_eigs[i,:] = vdata
        
    return norm_eigs

In [4]:
# Function to get eigen decomposition based on omega and speed
# import matplotlib.pyplot as plt

def eigmode_widget(C, D, fvec, num_ev, labels, f_in, speed, num_em):
    freq_index = np.abs(fvec-f_in).argmin()
    omega = 2*np.pi*fvec[freq_index]
    L, selected_Evec, sorted_Evals = get_complex_laplacian(C, D, w = omega, speed = speed, num_ev = num_ev)
    norm_eigs = normalize_eigs(selected_Evec)
    
    # Get our eigen modes (cortical only)
    cortical_norm_eigs = norm_eigs[:,0:68]
    lh_norm_eigs = cortical_norm_eigs[num_em-1,0:34]

    # need to pad our left hemisphere eigen vector so that len(ev) = len(names)
    #lh_norm_eigs[0,:]
    lh_padded_eigs = np.insert(lh_norm_eigs, [0, 3], [0, 0])
        
    vtx_test_brain = lh_padded_eigs[labels]
    brain.add_data(vtx_test_brain, hemi = 'lh', thresh = 0, colormap = "jet", remove_existing = True)
    brain.update_text(text = "Eig Number %s, frequency = %s, speed = %s" %(num_em, f_in, speed), name = "eiglabel")
    return norm_eigs

In [5]:
%gui qt

# set up Pysurfer variables
subject_id = "fsaverage"
hemi = "lh"
surf = "inflated"
"""
Bring up the visualization.
"""
#brain = Brain(subject_id, hemi , surf, background="white", views = ['lat','med'])
brain = surfBrain(subject_id, hemi, surf, background = "white", title = "Eigen Modes of Complex LaPlacian")
brain.add_text(x = 0.5, y = 0.9, text = "Eig Number , frequency = , speed = ", name = "eiglabel")

"""
Read in the automatic parcellation of sulci and gyri.
"""

hemi_side = "lh"
aparc_file = os.path.join(os.environ["SUBJECTS_DIR"],
                          subject_id, "label",
                          hemi_side + ".aparc.annot")
labels, ctab, names = nib.freesurfer.read_annot(aparc_file)

In [6]:
#%matplotlib inline
interactive(eigmode_widget, C = fixed(em_brain.reducedConnectome), D = fixed(em_brain.distance_matrix), fvec = fixed(fvec),
            num_ev = fixed(10), labels = fixed(labels),
            f_in = widgets.IntSlider(min=2,max=40,step=1,value=10, description = 'Frequency',continuous_update=False),
            speed = widgets.IntSlider(min = 5, max = 50, step = 5, value = 10, description = 'Speed',continuous_update=False),
            num_em = widgets.IntSlider(min = 1, max = 10, step = 1, value = 1, description = 'Eig Num',continuous_update=False))

interactive(children=(IntSlider(value=10, continuous_update=False, description='Frequency', max=40, min=2), In…

In [6]:
freq_index = np.abs(fvec-10).argmin()
omega = 2*np.pi*fvec[freq_index]
L, selected_Evec, sorted_Evals = get_complex_laplacian(em_brain.reducedConnectome, em_brain.distance_matrix, w = omega, speed = 10, num_ev = 10)
norm_eigs = normalize_eigs(selected_Evec)

In [7]:
norm_eigs.shape

(10, 86)

In [8]:
brain.add_data